# State Pattern

## Intent

> State is a behavioral design pattern that lets an object alter its behavior when its internal state changes. It appears as if the object changed its class. (refactoring.guru)

##  Applicability

當物件有多個可能的狀態且時常變動，並且物件的行為會根據狀態而有不同的表現。

## Structure

![structure](https://refactoring.guru/images/patterns/diagrams/state/structure-en.png)

圖片來源: 

## Example

以下是工單狀態的定義

> * PENDING (已排定): 工單尚未執行生產, 需要等待現場人員決定是否執行.
> * ACTIVE (進行中): 工單確定進行投產, 此時可以進行投料或收料.
> * CLOSING (待收料): 工單不再需要投入的材料, 作業人員僅需等待設備生產出半製品或製品, 並進行收料.
> * CLOSED (已完成): 工單已完成投入產出, 不再進行任何投料或收料行為.
> * SKIPPED (已跳過): 經現場人員判斷後, 該工單不需要執行.

### Without State Pattern

如果不考慮 state pattern 我們可能為這樣實作它

```go
type WorkOrderStatus int

const(
    Pending WorkOrderStatus = iota
    Active
    Closing
    Closed
    Skipped
)

type WorkOrder struct{
    Status WorkOrderStatus
    //...
}

// constructor
func NewWorkOrder()WorkOrder{
    return WorkOrder{
        Status: Pending,
        //...
    }
}

func(wo *WorkOrder)Feed(finish bool){
    switch wo.Status{
        case Pending:
            print("尚未執行的工單無法進行投料")
        case Active:
            print("進行投料")
            if finish{
                wo.Status = Closing
            }
        case Closing:
            print("待收料的工單無法進行投料")
        case Closed:
            print("已結束的工單無法進行投料")
        case Skipped:
            print("已取消的工單無法進行投料")
    }
}

//...
```

先把這個例子記下來，接著我們來看以`state pattern`實作它會變得如何。

### With State Pattern



In [ ]:
public class WorkOrder{
    private IWorkOrderStatus status;
    public WorkOrder(){
        status = new Pending(this);
    }
    public void ChangeStatus(IWorkOrderStatus status)=>this.status=status;
    public void Feed(bool finish) => status.Feed(finish);
    public void Collect() => status.Collect();
    public void Close() => status.Close();
    public void Skip() => status.Skip();
}

public interface IWorkOrderStatus{
    void Feed(bool finish);
    void Collect();
    void Close();
    void Skip();
}

public class Pending : IWorkOrderStatus{
    private WorkOrder wo;
    public Pending(WorkOrder wo){
        this.wo = wo;
    }
    public void Feed(bool finish){
        Console.WriteLine("未執行的工單無法進行投料");
    }
    public void Collect(){
        Console.WriteLine("未執行的工單無法進行收料");
    }
    public void Close(){
        Console.WriteLine("未執行的工單無法關閉");
    }
    public void Skip(){
        Console.WriteLine("跳過工單");
        wo.ChangeStatus(new Skipped(wo));
    }
}

public class Active : IWorkOrderStatus{
    private WorkOrder wo;
    public Active(WorkOrder wo){
        this.wo = wo;
    }
    public void Feed(bool finish){
        Console.WriteLine("進行投料");
        if(finish)
            wo.ChangeStatus(new Closing(wo));
    }
    public void Collect(){
        Console.WriteLine("進行收料");
    }
    public void Close(){
        Console.WriteLine("關閉工單");
        wo.ChangeStatus(new Closed(wo));
    }
    public void Skip(){
        Console.WriteLine("跳過工單");
        wo.ChangeStatus(new Skipped(wo));
    }
}

public class Closing : IWorkOrderStatus{
    private WorkOrder wo;
    public Closing(WorkOrder wo){
        this.wo = wo;
    }
    public void Feed(bool finish){
        Console.WriteLine("待收料工單無法進行投料");
    }
    public void Collect(){
        Console.WriteLine("進行收料");
    }
    public void Close(){
        Console.WriteLine("關閉工單");
        wo.ChangeStatus(new Closed(wo));
    }
    public void Skip(){
        Console.WriteLine("跳過工單");
        wo.ChangeStatus(new Skipped(wo));
    }
}

public class Closed : IWorkOrderStatus{
    private WorkOrder wo;
    public Closed(WorkOrder wo){
        this.wo = wo;
    }
    public void Feed(bool finish){
        Console.WriteLine("已關閉工單無法進行投料");
    }
    public void Collect(){
        Console.WriteLine("已關閉工單無法進行收料");
    }
    public void Close(){
        Console.WriteLine("已關閉工單無法再次關閉");
    }
    public void Skip(){
        Console.WriteLine("已關閉工單無法跳過");
    }
}

public class Skipped : IWorkOrderStatus{
    private WorkOrder wo;
    public Skipped(WorkOrder wo){
        this.wo = wo;
    }
    public void Feed(bool finish){
        Console.WriteLine("已跳過工單無法進行投料");
    }
    public void Collect(){
        Console.WriteLine("已跳過工單無法進行收料");
    }
    public void Close(){
        Console.WriteLine("已跳過工單無法關閉");
    }
    public void Skip(){
        Console.WriteLine("已跳過工單無法再次跳過");
    }
}

執行

In [ ]:
Console.WriteLine("=====>排定新工單");
WorkOrder workOrder = new();
Console.WriteLine("=====>投料");
workOrder.Feed(false);
Console.WriteLine("=====>執行工單");
workOrder.ChangeStatus(new Active(workOrder));
Console.WriteLine("=====>投料");
workOrder.Feed(false);
Console.WriteLine("=====>收料");
workOrder.Collect();
Console.WriteLine("=====>最後一次投料");
workOrder.Feed(true);
Console.WriteLine("=====>收料");
workOrder.Collect();
Console.WriteLine("=====>投料");
workOrder.Feed(false);
Console.WriteLine("=====>關閉");
workOrder.Close();
Console.WriteLine("=====>收料");
workOrder.Collect();
Console.WriteLine("=====>跳過");
workOrder.Skip();

#### Class Diagram

![cd](https://i.imgur.com/puZ6FQw.png)

### 變體

`Context`和`State`相互依賴令人感到莫名煩躁，試著改一下設計去解耦。

修改interface，在每個動作之後都回傳下一個狀態。這麼一來`State`就不需要依賴`Context`了。

In [ ]:
public class WorkOrder{
    private IWorkOrderStatus status = new Pending();
    public void Active()=>status = new Active();
    public void Feed(bool finish) {
        status = status.Feed(finish);
    }
    public void Collect() {
        status = status.Collect();
    }
    public void Close() {
        status = status.Close();
    }
    public void Skip() {
        status = status.Skip();
    }
}

public interface IWorkOrderStatus{
    IWorkOrderStatus Feed(bool finish);
    IWorkOrderStatus Collect();
    IWorkOrderStatus Close();
    IWorkOrderStatus Skip();
}

public class Pending : IWorkOrderStatus{
    public IWorkOrderStatus Feed(bool finish){
        Console.WriteLine("未執行的工單無法進行投料");
        return this;
    }
    public IWorkOrderStatus Collect(){
        Console.WriteLine("未執行的工單無法進行收料");
        return this;
    }
    public IWorkOrderStatus Close(){
        Console.WriteLine("未執行的工單無法關閉");
        return this;
    }
    public IWorkOrderStatus Skip(){
        Console.WriteLine("跳過工單");
        return Skip();
    }
}

public class Active : IWorkOrderStatus{
    public IWorkOrderStatus Feed(bool finish){
        Console.WriteLine("進行投料");
        return finish?new Closing():this;
    }
    public IWorkOrderStatus Collect(){
        Console.WriteLine("進行收料");
        return this;
    }
    public IWorkOrderStatus Close(){
        Console.WriteLine("關閉工單");
        return new Closed();
    }
    public IWorkOrderStatus Skip(){
        Console.WriteLine("跳過工單");
        return new Skipped();
    }
}

public class Closing : IWorkOrderStatus{
    public IWorkOrderStatus Feed(bool finish){
        Console.WriteLine("待收料工單無法進行投料");
        return this;
    }
    public IWorkOrderStatus Collect(){
        Console.WriteLine("進行收料");
        return this;
    }
    public IWorkOrderStatus Close(){
        Console.WriteLine("關閉工單");
        return new Closed();
    }
    public IWorkOrderStatus Skip(){
        Console.WriteLine("跳過工單");
        return new Skipped();
    }
}

public class Closed : IWorkOrderStatus{
    public IWorkOrderStatus Feed(bool finish){
        Console.WriteLine("已關閉工單無法進行投料");
        return this;
    }
    public IWorkOrderStatus Collect(){
        Console.WriteLine("已關閉工單無法進行收料");
        return this;
    }
    public IWorkOrderStatus Close(){
        Console.WriteLine("已關閉工單無法再次關閉");
        return this;
    }
    public IWorkOrderStatus Skip(){
        Console.WriteLine("已關閉工單無法跳過");
        return this;
    }
}

public class Skipped : IWorkOrderStatus{
    public IWorkOrderStatus Feed(bool finish){
        Console.WriteLine("已跳過工單無法進行投料");
        return this;
    }
    public IWorkOrderStatus Collect(){
        Console.WriteLine("已跳過工單無法進行收料");
        return this;
    }
    public IWorkOrderStatus Close(){
        Console.WriteLine("已跳過工單無法關閉");
        return this;
    }
    public IWorkOrderStatus Skip(){
        Console.WriteLine("已跳過工單無法再次跳過");
        return this;
    }
}

執行

In [ ]:
Console.WriteLine("=====>排定新工單");
WorkOrder workOrder = new();
Console.WriteLine("=====>投料");
workOrder.Feed(false);
Console.WriteLine("=====>執行工單");
workOrder.Active();
Console.WriteLine("=====>投料");
workOrder.Feed(false);
Console.WriteLine("=====>收料");
workOrder.Collect();
Console.WriteLine("=====>最後一次投料");
workOrder.Feed(true);
Console.WriteLine("=====>收料");
workOrder.Collect();
Console.WriteLine("=====>投料");
workOrder.Feed(false);
Console.WriteLine("=====>關閉");
workOrder.Close();
Console.WriteLine("=====>收料");
workOrder.Collect();
Console.WriteLine("=====>跳過");
workOrder.Skip();

#### Class Diagram

![cd](https://i.imgur.com/tUg9b7d.png)

## Reference

* [refactoring.guru](https://refactoring.guru/design-patterns/state)